In [13]:
import pandas as pd
import numpy as np
import warnings

from sklearn.metrics import precision_score
from sklearn.preprocessing import StandardScaler
from sklearn.neural_network import MLPClassifier

from sys import platform

if platform == "win32":
    path = 'C:/Users/olive/GitHub/f1-analytics/'
elif platform == "darwin":
    path = '~/Documents/GitHub/f1-analytics/'

warnings.filterwarnings("ignore", category=RuntimeWarning) 
pd.options.mode.chained_assignment = None  # default='warn'

%matplotlib inline

In [14]:
data = pd.read_csv(path+'data/ml_input.csv')
merged = pd.read_csv(path+'data/processed.csv')
parameters = pd.read_csv(path+'parameters/neutralnetworkclassifier.csv')

In [15]:
N = 2022

In [16]:
params_to_drop = ['season', 'round', 'podium', 'driver_points_from']

In [17]:
df = data.copy()
df.podium = df.podium.map(lambda x: 1 if x == 1 else 0)

train = df[df.season < N]
X_train = train.drop(params_to_drop, axis=1)
y_train = train.podium

scaler = StandardScaler()
X_train = pd.DataFrame(scaler.fit_transform(X_train), columns = X_train.columns)

In [18]:
data.columns

Index(['season', 'round', 'podium', 'q_delta', 'starting_grid',
       'driver_points_before', 'constructor_points_before',
       'driver_points_from', 'driver_points_per', 'points_percentage',
       'driver_last_3', 'constructor_last_3', 'stage_q1', 'stage_q2',
       'stage_q3'],
      dtype='object')

In [19]:
def get_predictions(model):
    predictions = []
    for circuit in df[df.season == N]['round'].unique():

        test = df[(df.season == N) & (df['round'] == circuit)]
        X_test = test.drop(params_to_drop, axis=1)
        y_test = test.podium
        
        #scaling
        X_test = pd.DataFrame(scaler.transform(X_test), columns = X_test.columns)
        
        # make predictions
        prediction_df = pd.DataFrame(model.predict_proba(X_test), columns=['proba_0', 'proba_1'])
        prediction_df['actual'] = y_test.reset_index(drop=True)
        prediction_df['predicted'] = prediction_df.proba_1.map(lambda x: 1 if float(x) == float(prediction_df.proba_1.max()) else 0)
        

        predictions += list(prediction_df.predicted.values)

    return predictions

In [20]:
parameters.head()

,Unnamed: 0,model,hidden_layer_sizes,activation,solver,alpha,score
0,360,neural_network_classifier,"(75, 25, 50, 10)",tanh,lbfgs,0.000100,0.545455
1,372,neural_network_classifier,"(75, 25, 50, 10)",tanh,lbfgs,0.615848,0.500000
2,194,neural_network_classifier,"(80, 20, 40, 5)",relu,lbfgs,2.636651,0.500000
3,366,neural_network_classifier,"(75, 25, 50, 10)",tanh,lbfgs,0.007848,0.500000
4,367,neural_network_classifier,"(75, 25, 50, 10)",tanh,lbfgs,0.016238,0.500000


In [21]:
params = parameters.iloc[1]
hidden_layer_sizes = eval(params.hidden_layer_sizes)
activation = params.activation
solver = params.solver
alpha = params.alpha

params

Unnamed: 0                                  372
model                 neural_network_classifier
hidden_layer_sizes             (75, 25, 50, 10)
activation                                 tanh
solver                                    lbfgs
alpha                                  0.615848
score                                       0.5
Name: 1, dtype: object

In [22]:
model = MLPClassifier(
    hidden_layer_sizes=hidden_layer_sizes,
    activation=activation, 
    solver=solver, alpha=alpha, 
    random_state=1)

model.fit(X_train, y_train)

predictions = get_predictions(model)

/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/sklearn/neural_network/_multilayer_perceptron.py:559: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  self.n_iter_ = _check_optimize_result("lbfgs", opt_res, self.max_iter)


In [23]:
test = merged.query('season ==@N')
test['predicted'] = predictions
test = test[['season', 'round', 'circuit_id', 'driver', 'starting_grid' ,'podium', 'predicted']]

In [24]:
test.query('predicted == 1')

,season,round,circuit_id,driver,starting_grid,podium,predicted
3111,2022,1,bahrain,charles_leclerc,1,1,1
3133,2022,2,jeddah,carlos_sainz,3,3,1
3149,2022,3,albert_park,charles_leclerc,1,1,1
3168,2022,5,miami,max_verstappen,3,1,1
3187,2022,6,catalunya,max_verstappen,2,1,1
3210,2022,7,monaco,charles_leclerc,1,4,1
3245,2022,8,baku,charles_leclerc,1,19,1
3247,2022,9,villeneuve,max_verstappen,1,1,1
3267,2022,10,silverstone,carlos_sainz,1,1,1
3305,2022,12,ricard,charles_leclerc,1,19,1
